# Pac-Man on Rails

In [41]:
#!pip install --user gtfs-realtime-bindings
#!pip install --user protobuf3-to-dict

In [2]:
import requests
import warnings
import zipfile
import pandas as pd
import ipywidgets as widgets
import google.transit.gtfs_realtime_pb2 as gtfs

In [22]:
from protobuf_to_dict import protobuf_to_dict
from ipywidgets import HTML, IntSlider, interact

In [4]:
warnings.filterwarnings('ignore')

## Static baseline

In [34]:
# Load static dataframes
static = []
names = []
with zipfile.ZipFile('data/gtfs_fp2020_2020-06-10.zip') as myzip:
    myzip.infolist()
    for filename in myzip.namelist():
        print('File:', filename)
        names.append(filename)
        df = pd.read_csv(myzip.open(filename))
        static.append(df)

File: agency.txt
File: routes.txt
File: trips.txt
File: stops.txt
File: calendar.txt
File: calendar_dates.txt
File: transfers.txt
File: feed_info.txt
File: stop_times.txt


In [35]:
@interact(i=(0, len(static) - 1))
def f(i):
    print(names[i])
    return static[i]

interactive(children=(IntSlider(value=4, description='i', max=8), Output()), _dom_classes=('widget-interact',)…

## Realtime feed

In [36]:
# Initialize feed
feed = gtfs.FeedMessage()

In [37]:
# Authorization header
headers = {
    "Accept": "application/octet-stream",
    "Authorization":
    "",
}

In [38]:
# Send request and retrieve response
response = requests.get("https://api.opentransportdata.swiss/gtfs-rt",
                        allow_redirects=True,
                        headers=headers)
# Validate response
response.reason

'OK'

In [39]:
# Parse feed
feed.ParseFromString(response.content)
# Number of trips in 3 hours window
len(feed.entity)

4139

In [40]:
@interact(i=(0, len(feed.entity) - 1))
def f(i):
    s = feed.entity[i]
    if s.HasField('trip_update'):
        print(s.trip_update.trip)

        @interact(j=(0, len(s.trip_update.stop_time_update) - 1))
        def g(j):
            print(s.trip_update.stop_time_update[j])

interactive(children=(IntSlider(value=2069, description='i', max=4138), Output()), _dom_classes=('widget-inter…